In [1]:
# Update : added test data to train for preprocessing 

In [2]:
# importing libraries
import pandas as pd
import numpy as np

In [3]:
#4.Complaint-reason to bag of words,tf-idf model
# Importing the libraries
import re
import pickle 
import nltk
#nltk.download('stopwords')
print('Download Stopwords')
from nltk.corpus import stopwords

Download Stopwords


In [4]:
# reading Excel file
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
submission = pd.read_csv('dataset/sample_submission.csv')

In [5]:
#Create Original files copy
train_org = train.copy()
test_org = test.copy()
submission_org = submission.copy()

In [6]:
test['Complaint-Status'] = np.nan
test = test[train.columns]

In [7]:
train = train.append(test)

In [8]:
############################################
#                                          #
#     2.1 Exploratory Data Analysis        #
#                                          #
############################################

###################################
#  2.1.1 understanding the data   #
###################################
#Checking few observation of dataset
#train.head()

In [9]:
###################################
#  2.1.2 Missing value analysis   #
###################################
    
# checking for missing values in dataset
for i in train.columns :
    print(i, train[i].isnull().sum())  

Complaint-ID 0
Date-received 0
Transaction-Type 0
Complaint-reason 0
Company-response 32207
Date-sent-to-company 0
Complaint-Status 18543
Consumer-disputes 11002
Consumer-complaint-summary 0


In [10]:
# counting of each unique values in each categorical variable
'''print("value counts of categorical column")
print()
for i in train.columns[0] :
    print('## {} ## :- '.format(i))
    #print(train[i].value_counts())
    #input()'''

'print("value counts of categorical column")\nprint()\nfor i in train.columns[0] :\n    print(\'## {} ## :- \'.format(i))\n    #print(train[i].value_counts())\n    #input()'

In [11]:

'''#####Observations
1.Convert Complian-Id to Index for obvious reasons
2.chk if Date-received == Date-sent-to-company;
Convert Date-received and Date-sent-to-company to year,month,day format
3.Trasaction Type , check frequency of categories
4.Complaint-reason to bag of words,tf-idf model
5.Check frequency of categories in taret variable - complaint-status, company-response,consumer-disputes
6.for consumer-complaint-summary, apply regex, follwed by bag of words,tf-idf conversion
8. investigate missing values in details, as 2columns have very high values
'''

'#####Observations\n1.Convert Complian-Id to Index for obvious reasons\n2.chk if Date-received == Date-sent-to-company;\nConvert Date-received and Date-sent-to-company to year,month,day format\n3.Trasaction Type , check frequency of categories\n4.Complaint-reason to bag of words,tf-idf model\n5.Check frequency of categories in taret variable - complaint-status, company-response,consumer-disputes\n6.for consumer-complaint-summary, apply regex, follwed by bag of words,tf-idf conversion\n8. investigate missing values in details, as 2columns have very high values\n'

In [12]:
# We have unique Complaint-ids and complain-id it shold not have any contribution
#1.Convert Complian-Id to Index for obvious reasons
train.set_index('Complaint-ID',inplace=True)

In [13]:
# 2.chk if Date-received == Date-sent-to-company;
#train['Date-received'] == train['Date-sent-to-company']
# so, we observe several false values , conclude dates are different

In [14]:
from datetime import datetime
train['Date-received'] = pd.to_datetime(train['Date-received'])
train['Date-sent-to-company'] = pd.to_datetime(train['Date-sent-to-company'])

In [15]:
# Delay in date-recived to date-sent-to-company should resemble comnplaint response or etc
train['Delay-dr-to-dstc'] = train['Date-sent-to-company']-train['Date-received']

In [16]:
'''#Convert Date-received and Date-sent-to-company to year,month,day format
#for i in train.index :
train['year-dr'] = train['Date-received'].dt.year
train['month-dr'] = train['Date-received'].dt.month
train['day-dr'] = train['Date-received'].dt.day
#train.loc[i,'day-dr'] = train.loc[i,'Date-received'].dt.day
train['year-ds2c'] = train['Date-sent-to-company'].dt.year
train['month-ds2c'] = train['Date-sent-to-company'].dt.month
train['day-ds2c'] = train['Date-sent-to-company'].dt.day'''

"#Convert Date-received and Date-sent-to-company to year,month,day format\n#for i in train.index :\ntrain['year-dr'] = train['Date-received'].dt.year\ntrain['month-dr'] = train['Date-received'].dt.month\ntrain['day-dr'] = train['Date-received'].dt.day\n#train.loc[i,'day-dr'] = train.loc[i,'Date-received'].dt.day\ntrain['year-ds2c'] = train['Date-sent-to-company'].dt.year\ntrain['month-ds2c'] = train['Date-sent-to-company'].dt.month\ntrain['day-ds2c'] = train['Date-sent-to-company'].dt.day"

In [17]:
train = train.drop(['Date-received','Date-sent-to-company'],axis=1)

In [18]:
'''temp = (train['day-dr'] == train['day-ds2c'])
temp.value_counts()'''

"temp = (train['day-dr'] == train['day-ds2c'])\ntemp.value_counts()"

In [19]:
train.head(4)

,Transaction-Type,Complaint-reason,Company-response,Complaint-Status,Consumer-disputes,Consumer-complaint-summary,Delay-dr-to-dstc
Complaint-ID,,,,,,,
Tr-1,Mortgage,"Loan servicing, payments, escrow account",NaN,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d...",0 days
Tr-2,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...,0 days
Tr-3,Bank account or service,Using a debit or ATM card,NaN,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel...",0 days
Tr-4,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea...",0 days


In [20]:
#find out target levels categorization
c = train['Complaint-Status'].astype('category')
rule = dict(enumerate(c.cat.categories))
print(rule)

{0: 'Closed', 1: 'Closed with explanation', 2: 'Closed with monetary relief', 3: 'Closed with non-monetary relief', 4: 'Untimely response'}


In [21]:
for col in ['Transaction-Type','Complaint-Status','Consumer-disputes']:
    train[col] = pd.Categorical(train[col])
    train[col] = train[col].cat.codes
for i in range(0,train.shape[1]):
    train.iloc[:,i] = train.iloc[:,].replace(-1,np.nan)

In [22]:
# 2.chk if Date-received == Date-sent-to-company;
#train['month-dr'] == train['month-ds2c']

In [23]:
train = train[['Complaint-Status','Transaction-Type', 'Complaint-reason', 'Company-response', 'Consumer-disputes', 'Consumer-complaint-summary','Delay-dr-to-dstc']]

In [ ]:
for col in ['Complaint-reason','Company-response','Consumer-complaint-summary']:
    for i in (train.index):
        train.loc[i,col] = re.sub(r'\W', ' ', str(train.loc[i,col]))
        train.loc[i,col] = train.loc[i,col].lower()
        train.loc[i,col] = re.sub(r'^br$', ' ', train.loc[i,col])
        train.loc[i,col] = re.sub(r'\s+br\s+',' ',train.loc[i,col])
        train.loc[i,col] = re.sub(r'\s+[a-z]\s+', ' ',train.loc[i,col])
        train.loc[i,col] = re.sub(r'^b\s+', '', train.loc[i,col])
        train.loc[i,col] = re.sub(r'\s+', ' ', train.loc[i,col])
print('breakpoint 1')

In [ ]:
train['Delay-dr-to-dstc'] = train['Delay-dr-to-dstc']/ np.timedelta64(1, 'D')

In [ ]:
#from fancyimpute import KNN
#train = pd.DataFrame(KNN(k=3).complete(train),columns = ['Transaction-Type','Complaint-Status','Consumer-disputes'])
train = train.fillna(0)

In [ ]:
train.to_csv('data_regex.csv')

In [ ]:
'''from sklearn import metrics
print(metrics.classification_report(ypred, ytest))'''

In [ ]:
'''# Creating the Tf-Idf Model
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X = transformer.fit_transform(X).toarray()'''